# Other attempts

This notebook addresses attempts that we tried as part of our project trajectory and decided not to include as part of the final solution, such as PCA analysis, predicting positions of track in the playlist, etc.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#PCA-on-Scaled-dataframe" data-toc-modified-id="PCA-on-Scaled-dataframe-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>PCA on Scaled dataframe</a></span></li><li><span><a href="#Models-for-predicting-position-of-track-in-playlist" data-toc-modified-id="Models-for-predicting-position-of-track-in-playlist-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Models for predicting position of track in playlist</a></span></li><li><span><a href="#Code-for-retrieving-genre-labels-from-Spotipy-API" data-toc-modified-id="Code-for-retrieving-genre-labels-from-Spotipy-API-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Code for retrieving genre labels from Spotipy API</a></span></li></ul></div>

### PCA on Scaled dataframe

TODO - Need to normalize dataset before inputting to PCA?

In [ ]:
pca = PCA().fit(new_scaled_df)
pca_2d = pca.transform(new_scaled_df)
pcaX_train_df = pd.DataFrame(pca_2d)

In [ ]:
list_of_audio_features = ['acousticness', 'energy', 'liveness', 'loudness', 'tempo', 'valence', 'danceability', 'speechiness']

In [ ]:
for column_plot in list_of_audio_features:
    
    cut_off = round(np.median(new_scaled_df[column_plot]), 4)
    df_y_train = pd.DataFrame(new_scaled_df[column_plot]).set_index(pcaX_train_df.index)
    
    plt.figure()
    plt.scatter(pcaX_train_df.loc[df_y_train[column_plot] < cut_off, 0], pcaX_train_df.loc[df_y_train[column_plot] < cut_off, 1], alpha=0.4, label= str(column_plot) + "<" + str(cut_off))
    plt.scatter(pcaX_train_df.loc[df_y_train[column_plot] >= cut_off, 0], pcaX_train_df.loc[df_y_train[column_plot] >= cut_off, 1], alpha=0.4, label= str(column_plot) + ">=" + str(cut_off))

    plt.xlabel("First PCA Component Vector")
    plt.ylabel("Second PCA Component Vector")
    plt.title("First two principal components: Results for binning " + str(column_plot))
    plt.legend();

In [ ]:
print("Variance explained by top two components", pca.explained_variance_ratio_[0:2])

In [ ]:
plt.figure(figsize=(20,10))
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlabel("Number of PCA components used")
plt.ylabel("Cumulative explained variability")
plt.title("Cumulative explained variability from PCA components");

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(pcaX_train_df)
y_kmeans = kmeans.predict(pcaX_train_df)

plt.scatter(pcaX_train_df[0], pcaX_train_df[1], c=y_kmeans, s=50, cmap='viridis')

centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=200, alpha=0.5)
plt.title("K-means clustering results on PCA (no. of clusters = 3)");

### Models for predicting position of track in playlist

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(new_scaled_df, songs_df['pos'], test_size=0.33, random_state=42)

In [ ]:
logreg = LogisticRegression(C=100000, fit_intercept=False, solver='lbfgs', max_iter=10000)
logreg.fit(X_train, y_train)
logreg.predict(X_train)
logreg.score(X_test, y_test)

In [ ]:
linearreg = LinearRegression()
linearreg.fit(X_train, y_train)
linearreg.score(X_test, y_test)

### Code for retrieving genre labels from Spotipy API

In [ ]:
first_dataframe['genre'] = ''
for i in range(3000): #len(first_dataframe['artist_uri'])
    first_dataframe.reset_index(drop=True, inplace=True)
    album_str = sp.artist(first_dataframe['artist_uri'][i])
    json_obj = json.dumps(album_str)
    if '[]' != str(json.loads(json_obj)['genres']):
        if 'classical' in str(json.loads(json_obj)['genres']):
            first_dataframe['genre'][i] = "Classical"
        elif 'soundtrack' in str(json.loads(json_obj)['genres']):
            first_dataframe['genre'][i] = "Soundtrack"
        else:
            first_dataframe['genre'][i] = json.loads(json_obj)['genres']
    else:
        first_dataframe.drop(i, inplace=True)

In [ ]:
first_dataframe.head()

In [ ]:
first_dataframe[first_dataframe.genre.apply(lambda x: 'pop' in x)]['genre']

TODO - take a picture of this!